<a href="https://colab.research.google.com/github/m0o0scar/nerfstudio/blob/main/colab/NeRFStudio_Build_Dependencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Mount Google Drive
%cd /content
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

/content
Mounted at /content/gdrive


In [ ]:
# @title Prepare
%cd /content/

!pip install --upgrade pip

# Create a new folder to store the built output
from datetime import datetime
folder_name = datetime.now().strftime('%Y%m%d-%H%M%S')
root = '/content/gdrive/MyDrive/colab/prebuilt/' + folder_name

# Create wheelhouse that we'll populate with pre-built wheels
!mkdir -p $root/tinycudann_and_nerfstudio
!mkdir -p $root/colmap

print(folder_name)

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
20230313-014335


In [ ]:
# @title Build TinyCuda
# Build the python bindings from tiny-cuda-nn and copy to wheelhouse
%cd /content/
!rm -rf tiny-cuda-nn
!git clone --recursive https://github.com/NVlabs/tiny-cuda-nn

%cd tiny-cuda-nn/bindings/torch
!python setup.py bdist_wheel
%cp dist/*.whl $root/tinycudann_and_nerfstudio

# Test installing tinycudann from wheelhouse
%cd /content/
!pip install --no-index --find-links=$root/tinycudann_and_nerfstudio tinycudann

/content
Cloning into 'tiny-cuda-nn'...
remote: Enumerating objects: 3773, done.
remote: Counting objects: 100% (3773/3773), done.
remote: Compressing objects: 100% (1024/1024), done.
remote: Total 3773 (delta 2431), reused 3644 (delta 2388), pack-reused 0
Receiving objects: 100% (3773/3773), 19.46 MiB | 23.06 MiB/s, done.
Resolving deltas: 100% (2431/2431), done.
Submodule 'dependencies/cutlass' (https://github.com/NVIDIA/cutlass) registered for path 'dependencies/cutlass'
Submodule 'dependencies/fmt' (https://github.com/fmtlib/fmt) registered for path 'dependencies/fmt'
Cloning into '/content/tiny-cuda-nn/dependencies/cutlass'...
remote: Enumerating objects: 19821, done.        
remote: Counting objects: 100% (2469/2469), done.        
remote: Compressing objects: 100% (869/869), done.        
remote: Total 19821 (delta 1616), reused 2320 (delta 1564), pack-reused 17352        
Receiving objects: 100% (19821/19821), 31.04 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (14238/14238),

In [ ]:
# @title Build nerfstudio
# Build nerfstudio into wheelhouse
%cd /content/
!rm -rf nerfstudio
!git clone https://github.com/m0o0scar/nerfstudio.git

%cd nerfstudio
!pip wheel . --wheel-dir $root/tinycudann_and_nerfstudio

# Test installing nerfstudio from wheelhouse
%cd /content/
!pip install --no-index --find-links=$root/tinycudann_and_nerfstudio nerfstudio

/content
Cloning into 'nerfstudio'...
remote: Enumerating objects: 13677, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 13677 (delta 115), reused 125 (delta 51), pack-reused 13425
Receiving objects: 100% (13677/13677), 68.12 MiB | 28.48 MiB/s, done.
Resolving deltas: 100% (9587/9587), done.
/content/nerfstudio
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/nerfstudio
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.7/136.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/

In [ ]:
# @title Build COLMAP
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    nvidia-cuda-toolkit \
    nvidia-cuda-toolkit-gcc

%cd /content/
!rm -rf colmap
!git clone https://github.com/colmap/colmap

%cd colmap
%mkdir build

%cd build
!cmake .. -GNinja -DCMAKE_CUDA_ARCHITECTURES='70'
!ninja
!sudo ninja install

# Backup build outputs to GDrive
%cd /content/
%mkdir -p $root/colmap/local/bin
%mkdir -p $root/colmap/local/lib
%mkdir -p $root/colmap/local/share/applications
%mkdir -p $root/colmap/local/include

%cp    /usr/local/bin/colmap                        $root/colmap/local/bin/
%cp -r /usr/local/lib/colmap                        $root/colmap/local/lib/
%cp    /usr/local/share/applications/COLMAP.desktop $root/colmap/local/share/applications/
%cp -r /usr/local/share/colmap/                     $root/colmap/local/share/
%cp -r /usr/local/include/colmap                    $root/colmap/local/include/

%cd $root/colmap
!zip -r local.zip ./local && rm -rf ./local

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-filesystem-dev is already the newest version (1.71.0.0ubuntu2).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.71.0.0ubuntu2).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.71.0.0ubuntu2).
libboost-system-dev set to manually installed.
libboost-graph-dev is already the newest version (1.71.0.0ubuntu2).
libboost-graph-dev set to manually installed.
libboost-test-dev is already the newest version (1.71.0.0ubuntu2).
libboost-test-dev set to manually installed.
build-essential is already the newest version (12.8ubuntu1.1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
git is already the newest version (1:2.25.1-1ubuntu3.10).
libsqlite3-dev is already the newest version (3.31.1-4ubuntu0.5).
libsqlite3-dev set to manually installed.
qtbase5-dev 

In [ ]:
# @title Update symbolic link
%cd /content/gdrive/MyDrive/colab/prebuilt
!rm -rf latest
!ln -s $folder_name latest

/content/gdrive/MyDrive/colab/prebuilt
